<a href="https://colab.research.google.com/github/seravee08/GPU-Computation-of-Persistent-Homology-for-Image-Data/blob/main/TopoGPU_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install CubicalRipser and TopoGPU
!pip install cripser
!pip install https://github.com/seravee08/GPU-Computation-of-Persistent-Homology-for-Image-Data/raw/main/releases/topogpu-0.1.0-cp312-cp312-linux_x86_64.whl

# Download data
!wget "https://raw.githubusercontent.com/seravee08/GPU-Computation-of-Persistent-Homology-for-Image-Data/main/data/aneurism_256x256x256_uint8.raw" \
     -O /content/sample_data/aneurism_256x256x256_uint8.raw
!wget "https://raw.githubusercontent.com/seravee08/GPU-Computation-of-Persistent-Homology-for-Image-Data/main/data/lobster_301x324x56_uint8.raw" \
     -O /content/sample_data/lobster_301x324x56_uint8.raw

  Using cached https://github.com/seravee08/GPU-Computation-of-Persistent-Homology-for-Image-Data/raw/main/releases/topogpu-0.1.0-cp312-cp312-linux_x86_64.whl (21.0 MB)
--2025-11-21 07:35:34--  https://raw.githubusercontent.com/seravee08/GPU-Computation-of-Persistent-Homology-for-Image-Data/main/data/aneurism_256x256x256_uint8.raw
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16777216 (16M) [application/octet-stream]
Saving to: ‘/content/sample_data/aneurism_256x256x256_uint8.raw’

/content/sample_dat 100%[===================>]  16.00M  --.-KB/s    in 0.09s   

2025-11-21 07:35:35 (171 MB/s) - ‘/content/sample_data/aneurism_256x256x256_uint8.raw’ saved [16777216/16777216]

--2025-11-21 07:35:35--  https://raw.githubusercontent.com/seravee08/GPU-Computation

In [2]:
# Example file comparing TopoGPU vs. Cubical Ripser
# If in Colab, hardware accelerator has to be GPU

import time
import numpy as np
import topogpu, tcripser, cripser

# ---------- helpers ----------
def read_raw(path, shape_xyz, np_dtype):
    with open(path, "rb") as f:
        arr = np.fromfile(f, dtype=np_dtype)
    return arr.reshape(shape_xyz)

def count_nontrivial(pairs):
    if pairs.size == 0:
        return 0
    return int(np.count_nonzero(pairs[:, 2] > pairs[:, 1]))

def print_help(prog: str) -> None:
    print(
        f"""
        Required arguments
          height – image height (uint), corresponds to -height.
          width – image width (uint), corresponds to -width.
          depth – image depth (uint), corresponds to -depth.
          dtype – voxel datatype, one of uchar, ushort, int, float (same as -datatype).

        Optional arguments
          bndmat_num – boundary-relation buffer size (like -bufSize).
                  Default is 2000. Increase this if you see buffer-size related errors.
          bx, by, bz – CUDA block size in each dimension (like -blockSize_x, -blockSize_y, -blockSize_z).
                  If all are left as 0, TopoGPU chooses a block size automatically.
                  If you set one, set all three; each must be a power of 2 and ≥ 2.
          deviceID – GPU device index. Use -1 (default) to let TopoGPU pick a device automatically;
                 use 0, 1, ... to select a specific GPU on multi-GPU systems.

        Once created, the returned object exposes methods:
          configure(height, width, depth) – reconfigure image size if needed.
          compute_PH_from_file(filename, dtype) – run persistent homology from a .raw file.
          compute_PH_from_array(array) - run persistent homology from numpy array
          get_pairNum() / get_results() – retrieve the number of pairs and the full persistence output.
        """,
        end=""
    )

# ---------- config ----------
filename1  = "/content/sample_data/aneurism_256x256x256_uint8.raw"
height1, width1, depth1 = 256, 256, 256
dtype_str  = "uchar"          # file datatype: 'uchar' | 'ushort' | 'int' | 'float'
bndmat_num = 2000


# ---------- TopoGPU (GPU) ----------
t0  = time.perf_counter()
tg1 = topogpu.create3D(height1, width1, depth1, dtype_str, bndmat_num)
tg1.configure(height1, width1, depth1)

# run TopoGPU from file
tg1.compute_PH_from_file(filename1, dtype_str)

t1 = time.perf_counter()
res_topogpu = tg1.get_results();
tg1.reset()
print(f"TopoGPU  | time: {(t1 - t0):.4f} s | nontrivial: {res_topogpu.shape[0]}")


# ---------- tcripser (CPU, T-construction) ----------
# Note: CubicalRipser expects a dense ndarray. Use (depth, height, width) as in your original sample.
t2  = time.perf_counter()
arr = read_raw(filename1, (depth1, height1, width1), np.uint8).astype(np.float32, copy=False)
res_tcripser = tcripser.computePH(arr, maxdim=3)
t3  = time.perf_counter()

print(f"tcripser | time: {(t3 - t2):.4f} s | nontrivial: {count_nontrivial(res_tcripser)}")

TopoGPU  | time: 1.7238 s | nontrivial: 34494
tcripser | time: 42.1354 s | nontrivial: 34494


In [3]:
# Vlidate the outpus of TopoGPU
from collections import Counter

def _safe_round(x: np.ndarray, decimals: int = 3):
    x = np.asarray(x, dtype=np.float64)

    # finite mask
    finite = np.isfinite(x)
    out = np.empty_like(x)

    # round only finite values; silence overflow inside
    if finite.any():
        with np.errstate(over='ignore', invalid='ignore'):
            out[finite] = np.round(x[finite], decimals=decimals)

    # keep non-finite (inf, nan) as-is
    out[~finite] = x[~finite]
    return out

def _normalize_topogpu(ph_topo: np.ndarray, decimals: int = 3):
    """
    TopoGPU format:
      dim, birth, death, bz, by, bx, dz, dy, dx, ...
    Only use (dim, birth, death), with floats rounded to `decimals`.
    """
    ph_topo = np.asarray(ph_topo, dtype=np.float64)

    dims  = ph_topo[:, 0].astype(int)
    birth = _safe_round(ph_topo[:, 1], decimals=decimals)
    death = _safe_round(ph_topo[:, 2], decimals=decimals)

    keys = [(int(d), float(b), float(de)) for d, b, de in zip(dims, birth, death)]
    return keys

def _normalize_tcripser(ph_tcri: np.ndarray, decimals: int = 3):
    """
    tcripser format:
      dim, birth, death, x1, y1, z1, x2, y2, z2
    - Remove trivial pairs: death <= birth.
    - Only use (dim, birth, death), with floats rounded to `decimals`.
    """
    ph_tcri = np.asarray(ph_tcri, dtype=np.float64)

    # keep only nontrivial pairs: death > birth
    nontrivial_mask = ph_tcri[:, 2] > ph_tcri[:, 1]
    ph_tcri = ph_tcri[nontrivial_mask]

    dims  = ph_tcri[:, 0].astype(int)
    birth = _safe_round(ph_tcri[:, 1], decimals=decimals)
    death = _safe_round(ph_tcri[:, 2], decimals=decimals)

    keys = [(int(d), float(b), float(de)) for d, b, de in zip(dims, birth, death)]
    return keys

def compare_topogpu_vs_tcripser(ph_topo: np.ndarray,
                                ph_tcri: np.ndarray,
                                decimals: int = 3):
    """
    Compare TopoGPU and tcripser PH outputs up to `decimals` digits.
    - Only compares (dim, birth, death).
    - tcripser trivial pairs (death <= birth) are removed before comparison.
    - Order of rows does not matter; comparison is multiset-based.

    Prints pairs that appear only in TopoGPU or only in tcripser.
    """
    topo_keys = _normalize_topogpu(ph_topo, decimals=decimals)
    tcri_keys = _normalize_tcripser(ph_tcri, decimals=decimals)

    topo_counter = Counter(topo_keys)
    tcri_counter = Counter(tcri_keys)

    only_topo = topo_counter - tcri_counter
    only_tcri = tcri_counter - topo_counter

    print("=== Pairs present in TopoGPU but not in tcripser (or with different multiplicity) ===")
    if not only_topo:
        print("None.")
    else:
        for key, count in only_topo.items():
            print(f"count={count}, (dim, birth, death)={key}")

    print("\n=== Pairs present in tcripser but not in TopoGPU (or with different multiplicity) ===")
    if not only_tcri:
        print("None.")
    else:
        for key, count in only_tcri.items():
            print(f"count={count}, (dim, birth, death)={key}")

compare_topogpu_vs_tcripser(res_topogpu, res_tcripser, decimals=3)

=== Pairs present in TopoGPU but not in tcripser (or with different multiplicity) ===
None.

=== Pairs present in tcripser but not in TopoGPU (or with different multiplicity) ===
None.


In [4]:
# ---------- TopoGPU: batch mode ----------
filename2 = "/content/sample_data/lobster_301x324x56_uint8.raw"
height2, width2, depth2 = 324, 301, 56

# create TopoGPU with maximum height, width, depth of the dataset and manually set blockSize (8, 8, 8)
maxHeight, maxWidth, maxDepth = 512, 512, 512
tg2 = topogpu.create3D(maxHeight, maxWidth, maxDepth, dtype_str, bndmat_num, 8, 8, 8)

# compute for file1
tg2.configure(height1, width1, depth1)
arr = read_raw(filename1, (depth1, height1, width1), np.uint8)
tg2.compute_PH_from_array(arr) # run TopoGPU from numpy array
print(f"TopoGPU nontrivial: {tg2.get_results().shape[0]}")
tg2.reset();

# compute for file2
tg2.configure(height2, width2, depth2)
arr = read_raw(filename2, (depth2, height2, width2), np.uint8)
tg2.compute_PH_from_array(arr) # run TopoGPU from numpy array
print(f"TopoGPU nontrivial: {tg2.get_results().shape[0]}")
tg2.reset();

TopoGPU nontrivial: 34494
TopoGPU nontrivial: 140275
